# OCR 
- Tesseract Model:
training Google Tesseract pre-train model with 50 Vietnamese fonts
- Pre-processing:
using "imgtxtenh" library to preprocess an image. The function of 'imgtxtenh' is to clean/enhance noisy scanned text images.
- Image segmentation:
spliting an image into many rows of text. Inheriting the result of CRAFT model and G-DBSCAN(clustering algorithm)

# Pre-processing:
- Install imgtxtenh following this link: https://github.com/mauvilsa/imgtxtenh
- Run command: ./imgtxtenh -p [input_image]  [output_image]

# Image segmentation:

Notebook: from beginning to clustering stage

# Image to text:
Notebook: prediction stage


# Trình tự
- Tiền xử lý trên toàn bộ bức ảnh gốc với imgtxtenh để lọc nhiễu và background
- Khởi tạo thông số  cho craft model
- Dùng craft pretrain model dự đoán vị trí các từ trong ảnh sau khi lọc nhiễu, xoay ảnh nếu cần thiết
- Nhóm các từ theo hàng với G-DBSCAN(lưu ý 2 thông số xác định vùng lân cận của giải thuật)
- Bóc tách từng dữ liệu theo hàng, tiền xử lý đồng thời dùng tesseract model chuyển từng dòng ảnh sang chữ viết

Step-by-step:

###Step 1: Load image

###Step 2: Rotate image (or not)

###Step 3: Get the word bounding boxes

    • Input: Ảnh màu RGB, có width, height, channel

    • Output: Heat map và tọa độ bounding box của word

###Step 4: Lưu kết quả là hình ảnh đã có bounding box vào đường dẫn thư mục

###Step 5: Tính tọa độ của tâm các bounding box của word (Polys)

    • Input: Các Polys đã xác định được từ Step 4

    • Output: Tọa độ Central point của từng Polys

###Step 6: Chuyển mỗi tọa độ Central Point thành một Object Point, rồi đẩy vào mảng X

###Step 7: Reset lại 4 initial parameters của CRAFT (nếu  chạy nhiều ảnh)

###Step 8: DBScan từng Point trong Step 7 

  • Input: Các Polys đã xác định được từ Step 4

  • Output: Gom Point vào các Cluster (các word chung 1 hàng vào chung 1 cluster)

###Step 9: Với mỗi cluster tìm được trong Step 8, tạo 1 bounding box (Mỗi hàng text tạo 1 bounding box)

  • Output: list chứa các tọa độ của mỗi bounding box vừa tạo 
    (top_left, bottom_left, top_right,bottom_right)

###Step 10: Lưu kết quả là ảnh đã có bounding box vào đường dẫn thư mục

###Step 11: In ra nội dung text (dưới dạng chữ đánh máy) đã nhận diện được từ ảnh




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd "/content/drive/My Drive/OCR-PIPELINE/ocr_demo_code"

In [ ]:
!pip install pyproj
!pip install pytesseract
!sudo apt install tesseract-ocr

In [ ]:
import sys
import os
import time
import argparse
import getopt

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

from PIL import Image

import cv2
from skimage import io
import numpy as np
import ocr_demo_code.craft_utils
import ocr_demo_code.imgproc
import ocr_demo_code.file_utils
import vn2k_to_wgs84.vn2k_to_wgs84

from ocr_demo_code.craft import CRAFT
# OrderedDict: dictionary subclass that remembers the order that keys were first inserted
from collections import OrderedDict
import copy
import math

#import imutils
#import matplotlib.pyplot as plt
import pytesseract

#import webbrowser


In [ ]:
UNCLASSIFIED = -2
NOISE = -1


# CRAFT  

In [ ]:

def copyStateDict(state_dict):
    if list(state_dict.keys())[0].startswith("module"):
        start_idx = 1
    else:
        start_idx = 0
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = ".".join(k.split(".")[start_idx:])
        new_state_dict[name] = v
    return new_state_dict

def str2bool(v):
    return v.lower() in ("yes", "y", "true", "t", "1")

In [ ]:
def test_net(net, canvas_size, mag_ratio, image, text_threshold, link_threshold, low_text, cuda, poly, refine_net=None):
    t0 = time.time()
    # resize
    img_resized, target_ratio, size_heatmap = imgproc.resize_aspect_ratio(image, canvas_size, interpolation=cv2.INTER_LINEAR, mag_ratio=mag_ratio)
    ratio_h = ratio_w = 1 / target_ratio

    # preprocessing
    x = imgproc.normalizeMeanVariance(img_resized)
    x = torch.from_numpy(x).permute(2, 0, 1)    # [h, w, c] to [c, h, w]
    x = Variable(x.unsqueeze(0))                # [c, h, w] to [b, c, h, w]
    # print("X: ",x)
  
    if cuda:
        x = x.cuda()

    # forward pass
    with torch.no_grad():
        y, feature = net(x)

    # make score and link map
    score_text = y[0,:,:,0].cpu().data.numpy()
    score_link = y[0,:,:,1].cpu().data.numpy()
    # print("Score_text: ", score_text)
    # print("Score_link: ", score_link)

    # refine link
    if refine_net is not None:
        with torch.no_grad():
            y_refiner = refine_net(y, feature)
        score_link = y_refiner[0,:,:,0].cpu().data.numpy()
    t0 = time.time() - t0
    t1 = time.time()

    # Post-processing
    boxes, polys = craft_utils.getDetBoxes(score_text, score_link, text_threshold, link_threshold, low_text, poly)

    # coordinate adjustment
    boxes = craft_utils.adjustResultCoordinates(boxes, ratio_w, ratio_h)
    polys = craft_utils.adjustResultCoordinates(polys, ratio_w, ratio_h)
    for k in range(len(polys)):
        if polys[k] is None: polys[k] = boxes[k]
    t1 = time.time() - t1

    # render results (optional)
    render_img = score_text.copy()
    render_img = np.hstack((render_img, score_link))
    ret_score_text = imgproc.cvt2HeatmapImg(render_img)
    # print("Render image: ", render_img)
    # plt.imshow(ret_score_text)
    # print("Bounding Box: ", polys)

    # if show_time : print("\ninfer/postproc time : {:.3f}/{:.3f}".format(t0, t1))

    return boxes, polys, ret_score_text

# G-DBSCAN

In [ ]:
class Point:
    '''
    Each point have 2 main values: coordinate(lat, long) and cluster_id
    '''
    def __init__(self, x, y, id):
        self.x = x
        self.y = y
        self.id = id
        self.cluster_id = UNCLASSIFIED

    def __repr__(self):
        return '(x:{}, y:{}, id:{}, cluster:{})' \
            .format(self.x, self.y, self.id, self.cluster_id)

#In G-DBScan we use elip instead of circle to cluster (because we mainly use for horizontal text image --> elip is more useful)
def n_pred(p1, p2):
#     return (p1.x - p2.x)**2/160000 + (p1.y - p2.y)**2/2500 <= 1
#     print(p1.x -p2.x)
#     print(p1.y -p2.y)
#     return (p1.x - p2.x)**2/50000 + (p1.y - p2.y)**2/1500 <= 1
#     return (p1.x - p2.x)**2/20000 + (p1.y - p2.y)**2/1300 <= 1
#     return (p1.x - p2.x)**2/2000 + (p1.y - p2.y)**2/130 <= 1
      return (p1.x - p2.x)**2/500 + (p1.y - p2.y)**2/70 <= 1
#     return (p1.x - p2.x)**2/3500 + (p1.y - p2.y)**2/150 <= 1
#     return (p1.x - p2.x)**2/7000 + (p1.y - p2.y)**2/1300 <= 1
#     return (p1.x - p2.x)**2/8000 + (p1.y - p2.y)**2/300 <= 1
#     return (p1.x - p2.x)**2/17000 + (p1.y - p2.y)**2/300 <= 1
#     return (p1.x - p2.x)**2/13000 + (p1.y - p2.y)**2/250 <= 1
#    return (p1.x - p2.x)**2/15000 + (p1.y - p2.y)**2/180 <= 1


def w_card(points):
    return len(points)

In [ ]:
def GDBSCAN(points, n_pred, min_card, w_card):
    points = copy.deepcopy(points)
    cluster_id = 0
    for point in points:
        if point.cluster_id == UNCLASSIFIED:
            if _expand_cluster(points, point, cluster_id, n_pred, min_card,
                               w_card):
                cluster_id = cluster_id + 1
    clusters = {}
    for point in points:
        key = point.cluster_id
        if key in clusters:
            clusters[key].append(point)
        else:
            clusters[key] = [point]
    return list(clusters.values())


def _expand_cluster(points, point, cluster_id, n_pred, min_card, w_card):
    if not _in_selection(w_card, point):
        points.change_cluster_id(point, UNCLASSIFIED)
        return False

    seeds = points.neighborhood(point, n_pred)
    if not _core_point(w_card, min_card, seeds):
        points.change_cluster_id(point, NOISE)
        return False

    points.change_cluster_ids(seeds, cluster_id)
    seeds.remove(point)

    while len(seeds) > 0:
        current_point = seeds[0]
        result = points.neighborhood(current_point, n_pred)
        if w_card(result) >= min_card:
            for p in result:
                if w_card([p]) > 0 and p.cluster_id in [UNCLASSIFIED, NOISE]:
                    if p.cluster_id == UNCLASSIFIED:
                        seeds.append(p)
                    points.change_cluster_id(p, cluster_id)
        seeds.remove(current_point)
    return True


def _in_selection(w_card, point):
    return w_card([point]) > 0


def _core_point(w_card, min_card, points):
    return w_card(points) >= min_card


class Points:
    'Contain list of Point'
    def __init__(self, points):
        self.points = points

    def __iter__(self):
        for point in self.points:
            yield point

    def __repr__(self):
        return str(self.points)

    def get(self, index):
        return self.points[index]

    def neighborhood(self, point, n_pred):
        return list(filter(lambda x: n_pred(point, x), self.points))

    def change_cluster_ids(self, points, value):
        for point in points:
            self.change_cluster_id(point, value)

    def change_cluster_id(self, point, value):
        index = (self.points).index(point)
        self.points[index].cluster_id = value

    def labels(self):
        return set(map(lambda x: x.cluster_id, self.points))

# Word detection

Load CRAFT pre-train model

Load data and apply CRAFT model to recognize word blocks on an image

Link github of Craft Model: https://github.com/clovaai/CRAFT-pytorch
(Paper of Craft model we used also in this git link)

In [ ]:
# If you need rotate an image, use imutils library and call rotate method with specified number of angle  degrees about the center of the image
#In this case we don't need to rotate --> Just skip this step
import imutils
%matplotlib inline
import matplotlib.pyplot as plt

image = imutils.rotate(image, 90)
plt.imshow(image)
plt.show()

# Prediction stage
Use tesseract model to convert these segments of an image to text


In [ ]:
def img_to_text(image, image_path):
    # Initialize CRAFT parameters
    text_threshold = 0.7
    low_text = 0.4
    link_threshold = 0.4
    cuda = False
    canvas_size = 1280
    mag_ratio = 1.5
    # if text image present curve --> poly=true
    poly = False
    refine = False
    show_time = False
    refine_net = None
    trained_model_path = './craft_mlt_25k.pth'
    #trained_model_path = './vgg16.ckpt'

    net = CRAFT()
    net.load_state_dict(copyStateDict(torch.load(
        trained_model_path, map_location='cpu')))
    net.eval()

    # Apply predict function to the image then get the word bounding boxes
    # Use result of pre-trained Craft model to detect word in image by create bounding box for each word
    bboxes, polys, score_text = test_net(
        net, canvas_size, mag_ratio, image, text_threshold, link_threshold, low_text, cuda, poly, refine_net)

    file_utils.saveResult(
        image_path, image[:, :, ::-1], bboxes, dirname='./craft_result/')

    # Compute coordinate of central point in each bounding box returned by CRAFT
    # Purpose: easier for us to make cluster in G-DBScan step
    poly_indexes = {}
    central_poly_indexes = []
    for i in range(len(polys)):
        poly_indexes[i] = polys[i]
        x_central = (polys[i][0][0] + polys[i][1][0] +
                     polys[i][2][0] + polys[i][3][0])/4
        y_central = (polys[i][0][1] + polys[i][1][1] +
                     polys[i][2][1] + polys[i][3][1])/4
        central_poly_indexes.append({i: [int(x_central), int(y_central)]})

    X = []
    for idx, x in enumerate(central_poly_indexes):
        point = Point(x[idx][0], x[idx][1], idx)
        X.append(point)

    poly = False
    refine = False
    show_time = False
    refine_net = None

    clustered = GDBSCAN(Points(X), n_pred, 1, w_card)

    cluster_values = []
    for cluster in clustered:
        sort_cluster = sorted(cluster, key=lambda elem: (elem.x, elem.y))
        max_point_id = sort_cluster[len(sort_cluster) - 1].id
        min_point_id = sort_cluster[0].id
        max_rectangle = sorted(
            poly_indexes[max_point_id], key=lambda elem: (elem[0], elem[1]))
        min_rectangle = sorted(
            poly_indexes[min_point_id], key=lambda elem: (elem[0], elem[1]))

        right_above_max_vertex = max_rectangle[len(max_rectangle) - 1]
        right_below_max_vertex = max_rectangle[len(max_rectangle) - 2]
        left_above_min_vertex = min_rectangle[0]
        left_below_min_vertex = min_rectangle[1]

        if (int(min_rectangle[0][1]) > int(min_rectangle[1][1])):
            left_above_min_vertex = min_rectangle[1]
            left_below_min_vertex = min_rectangle[0]
        if (int(max_rectangle[len(max_rectangle) - 1][1]) < int(max_rectangle[len(max_rectangle) - 2][1])):
            right_above_max_vertex = max_rectangle[len(max_rectangle) - 2]
            right_below_max_vertex = max_rectangle[len(max_rectangle) - 1]

        cluster_values.append([left_above_min_vertex, left_below_min_vertex,
                               right_above_max_vertex, right_below_max_vertex])

    file_utils.saveResult(
        image_path, image[:, :, ::-1], cluster_values, dirname='./cluster_result/')

    img = np.array(image[:, :, ::-1])

    res = []
    for i, box in enumerate(cluster_values):
        poly = np.array(box).astype(np.int32).reshape((-1))
        poly = poly.reshape(-1, 2)

        rect = cv2.boundingRect(poly)
        x, y, w, h = rect
        croped = img[y:y+h, x:x+w].copy()

        # Preprocess croped segment
        croped = cv2.resize(croped, None, fx=5, fy=5,
                            interpolation=cv2.INTER_LINEAR)
        croped = cv2.cvtColor(croped, cv2.COLOR_BGR2GRAY)
        croped = cv2.GaussianBlur(croped, (3, 3), 0)
        croped = cv2.bilateralFilter(croped, 5, 25, 25)
        croped = cv2.dilate(croped, None, iterations=1)
        croped = cv2.threshold(
            croped, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        # croped = cv2.threshold(croped, 90, 255, cv2.THRESH_BINARY)[1]
        croped = cv2.cvtColor(croped, cv2.COLOR_BGR2RGB)

        text = pytesseract.image_to_string(croped, lang='eng')
        res.append(text)
    return res
  

In [ ]:
def X_Y_localization(listText):
  new_text = []
  res1 = []
  res2 = []
  count = {}
  for text in listText:
    # obmit space and endl in text
    text_without_space_endl = ''
    for word in text.split():
      text_without_space_endl += word
    
    if len(text_without_space_endl) < 4 or len(text_without_space_endl) > 11:
      continue
    # modify some easy to misunderstanding features in the text
    norm_text = ''
    for i in range(len(text_without_space_endl)):
        if text_without_space_endl[i] in ['S','$']:
          norm_text += '5'
        else:
          norm_text += text_without_space_endl[i]
    new_text.append(norm_text)
    # Check the frequency of first 4 letters in norm_text
    first_4 = norm_text[0:4]  
    if first_4 in count:
      count[first_4] += 1
    else:
      count[first_4] = 1  
  max = 0
  str1 = ''
  for i in count:
    if count[i] > max:
      max = count[i]
      str1 = i
  max = 0
  str2 = ''
  for i in count:
    if i != str1 and count[i] > max:
      max = count[i]
      str2 = i
  if not str1.isdigit() or not str2.isdigit():  # 4 so dau tien cuc ki quan trong, neu k xac dinh chinh xac 1 trong 2 thi xem nhu khong tim dc
    return res1, res2   # res1 = res2 = [] 
  avg_len1 = 0
  avg_len2 = 0
  for text in new_text:
    if text[0:4] == str1:
      res1.append(text)
      if '.' in text:
        avg_len1 += len(text)
      else:
        avg_len1 += len(text) + 1
    elif text[0:4] == str2:
      res2.append(text)
      if '.' in text:
        avg_len2 += len(text)
      else:
        avg_len2 += len(text) + 1
  avg_len1 = avg_len1 / len(res1)
  avg_len2 = avg_len2 / len(res2)
  if avg_len1 > avg_len2:
    return res1, res2
  else:
    return res2, res1

In [ ]:
def norm_X_Y(listX, listY):
  if listX == [] or listY == []:
    print('Cannot read X,Y from img')
    return [],[]
  float_listX = []
  float_listY = []
  for text in listX:
    if len(text) < 7:
      continue
    norm_text = ''
    if text.count('.') > 1 or ('.' in text and text.index('.') != 7):
      # neu xhien >1 '.' or dau '.' nam sai vi tri (sau chu so thu 7) thi bo toan bo '.' roi them lai sau
      for letter in text:
        if letter != '.':
          norm_text += letter
      text = norm_text
      norm_text = ''
    for i in range(len(text)):
      if text[i] == ',':
        norm_text += '.'
      elif text[i] != '.' and not text[i].isdigit():
        norm_text += '5'  # Vi cac so sau sai lech khong anh huong qua lon den toa do nen chuan hoa cac ki tu k phai so thanh 5
      else:
        norm_text += text[i]
    # Xu ly cac truong hop khong co '.'
    if norm_text.count('.') == 0:
      text = ''
      for i in range(len(norm_text)):
        if i == 6:
          text += norm_text[i] + '.'
        else:
          text += norm_text[i]
    else:
      text = norm_text
    float_listX.append(float(text))
  
  for text in listY:
    if len(text) < 6:
      continue
    norm_text = ''
    if text.count('.') > 1 or ('.' in text and text.index('.') != 6): 
      # neu xhien >1 '.' or dau '.' nam sai vi tri (sau chu so thu 6) thi bo toan bo '.' roi them lai sau
      for letter in text:
        if letter != '.':
          norm_text += letter
      text = norm_text
      norm_text = ''
    for i in range(len(text)):
      if text[i] == ',':
        norm_text += '.'
      elif text[i] != '.' and not text[i].isdigit():
        norm_text += '5'  # Vi cac so sau sai lech khong anh huong qua lon den toa do nen chuan hoa cac ki tu k phai so thanh 5
      else:
        norm_text += text[i]
    # Xu ly cac truong hop khong co '.'
    if norm_text.count('.') == 0:
      text = ''
      for i in range(len(norm_text)):
        if i == 5:
          text += norm_text[i] + '.'
        else:
          text += norm_text[i]
    else:
      text = norm_text
    float_listY.append(float(text))
  return float_listX, float_listY

In [ ]:
def get_X_Y (X,Y):
  res_x = 0
  res_y = 0
  if X == [] or Y == []:
    print('Cannot get X,Y. Try another img')
    return res_x, res_y
  found_x = 0
  found_y = 0
  for x in X:
    for another_x in X:
      if abs(another_x - x) > 0 and abs(another_x - x) < 10:
        found_x += 1
        res_x = (another_x + x) / 2
        break
    if found_x > 0:
      break

  for y in Y:
    for another_y in Y:
      if abs(another_y - y) > 0 and abs(another_y - y) < 10:
        found_y += 1
        res_y = (another_y + y) / 2
        break
    if found_y > 0:
      break
    
  if found_x == 0: # lay trung binh cong
    for x in X:
      res_x += x
    res_x = res_x / len(X)
  if found_y == 0: # lay trung binh cong
    for y in Y:
      res_y += y
    res_y = res_y / len(Y)
  return res_x, res_y

In [ ]:
def main(argv):
    inputfile = ''
    outputfile = ''
    try:
        opts, args = getopt.getopt(argv, "i:", ["ifile="])
    except getopt.GetoptError:
        print('Error!')
        sys.exit(2)
    for opt, arg in opts:
        if opt in ("-i", "--ifile"):
            inputfile = arg
    image_path = inputfile
    image = imgproc.loadImage(image_path)
   
    """
    fig2 = plt.figure(figsize=(10, 10))  # create a 10 x 10 figure
    ax3 = fig2.add_subplot(111)
    ax3.imshow(image, interpolation='none')
    ax3.set_title('larger figure')
    plt.show()
    """

    listText = img_to_text(image, image_path)
    print(listText)
    listX, listY = X_Y_localization(listText)
    print(listX, listY)
    X, Y = norm_X_Y(listX, listY)
    print(X, Y)
    mid_X, mid_Y = get_X_Y(X, Y)
    print(mid_X, mid_Y)
    new_X, new_Y = vn2k_to_wgs84.vn2k_to_wgs84((mid_X, mid_Y), 9210) # 9205 ~ 9218, 5896 ~ 5899
    print(new_X, new_Y)

    new_X = float(int(new_X * 10000000) / 10000000)
    new_Y = float(int(new_Y * 10000000) / 10000000)
    url = 'https://www.google.com/maps/@?api=1&map_action=map&center=' + str(new_Y) + ',' + str(new_X) + '&zoom=15'

    """
    ! Not working on colab
    webbrowser.open(url)
    """

    print(url)

In [ ]:
main(["-i", "./input_data/pre_Page_1.jpg.jpg"])

### This allows the program to be called from the command line
Does not work in notebook

In [ ]:
if __name__ == "__main__":
    # * Allows taking arguments from command line
    # ls | xargs --replace=$ python post.py -i $ -o post_$
    if len(sys.argv) < 2 or sys.argv[1] in ('-h', '--help'):
        print(
            'Try \'python {} -i <inputfile>\'.'.format(sys.argv[0]))
        print(
            'For bulk use \'ls | xargs --replace=$ python {} -i $\'.'.format(sys.argv[0]))
        sys.exit()
    else:
        main(sys.argv[1:])
